In [1]:
from stocks import csv2df, get_static, get_detail, get_summary


stocks_cny = csv2df('stocks-cny.csv', 'trades-cny.csv', 'dividends-cny.csv')
stocks_hkd = csv2df('stocks-hkd.csv', 'trades-hkd.csv', 'dividends-hkd.csv')
stocks_usd = csv2df('stocks-usd.csv', 'trades-usd.csv', 'dividends-usd.csv')

detail_cny = get_static(stocks_cny[0], stocks_cny[1], stocks_cny[2])
detail_hkd = get_static(stocks_hkd[0], stocks_hkd[1], stocks_hkd[2])
detail_usd = get_static(stocks_usd[0], stocks_usd[1], stocks_usd[2])

In [2]:
summary_cny = get_summary(detail_cny)
summary_hkd = get_summary(detail_hkd)
summary_usd = get_summary(detail_usd)

In [3]:
summary_cny

,Symbol,Name,Qty,Last,Earning,Return
5,600660,福耀玻璃,3000,28.66,52624.97,0.44
21,600585,海螺水泥,6000,28.34,38093.22,0.20
22,601939,建设银行,15000,6.99,21945.37,0.22
23,000895,双汇发展,6000,26.78,30379.95,0.12
27,601318,中国平安,3000,75.27,161860.71,0.42
30,501021,香港中小,117500,1.41,38682.50,0.30
38,000001,平安银行,20000,13.18,84781.64,0.47
50,000961,中南建设,100000,5.99,-39291.40,-0.06


In [4]:
summary_hkd

,Symbol,Name,Qty,Last,Earning,Return
3,01988,民生银行,12000,7.51,-3924.67,-0.02
8,00512,远大医药,32000,3.35,40235.55,0.43
31,01918,融创中国,15000,35.45,1273907.84,0.86
38,02601,中国太保,6000,38.95,27665.38,0.08
39,02666,环球医疗,20000,7.53,16394.48,0.12
45,00392,北京控股,5000,44.65,21917.79,0.11
50,00656,复星国际,8000,17.06,-12236.74,-0.05
51,00735,中国电力新能源,35000,4.47,-7690.87,-0.05
59,03883,中国奥园,40000,4.35,-5497.91,-0.03
61,00558,力劲科技,100000,1.30,31859.47,0.32
